In [2]:
from datetime import datetime
import sys
import traceback
import numpy as np
from datetime import datetime
import sys
import traceback
import scipy.io as sio
import torch

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
from scipy.io import savemat
import numpy as np
import math
import random
import time
from collections import OrderedDict
import matplotlib.pyplot as plt
from itertools import cycle
import itertools as it
from pathlib import Path
import os
#import mysql.connector as mysql

In [4]:
import torch
import torch.nn as nn
from collections import OrderedDict


class ConvBN(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size,groups=1):
        if not isinstance(kernel_size, int):
            padding = [(i - 1) // 2 for i in kernel_size]
        else:
            padding = (kernel_size - 1) // 2
        super(ConvBN, self).__init__(OrderedDict([
            ('conv', nn.Conv2d(in_planes, out_planes, kernel_size,
                               padding=padding)),
            ('bn', nn.BatchNorm2d(out_planes,eps=1e-03, momentum=0.99))
        ]))

        
        
class RefineNet(nn.Module):
    def __init__(self, img_channels=2):
        super(RefineNet, self).__init__()
        self.conv=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(img_channels,8,[1,7])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(8,16,[1,7])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,2,[1,7])),    
        ]))
        
        self.conv_1=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(img_channels,8,[1,5])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(8,16,[1,5])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,2, [1,5])),    
        ]))
        
        self.conv1x1 = ConvBN(4, 2, [1,7])
        self.Relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)
        
        
    def forward(self, x):
        ori_x = x.clone()

        # concatenate
        x_1 = self.conv(x)
        x_2 = self.conv_1(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.Relu(x)
        x =self.conv1x1(x)


        return self.Relu(x + ori_x)
    
    
    
    
class Encoder_Compression(nn.Module):
    def __init__(self):
        super(Encoder_Compression, self).__init__()
        self.conv=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(64,32,[1,7])),
            ("PReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_Conv1x7",ConvBN(32,16,[1,7])),
            ("PReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_Conv1x7",ConvBN(16,8,[1,7])),
        ]))
        
        self.conv_2=ConvBN(64,8,[1,7])
        self.conv_3=ConvBN(16,8,[1,7])
        
        
        self.LeakyRelu = nn.LeakyReLU(negative_slope=0.3, inplace=True)
               
    def forward(self, x):        
        x_1= self.conv(x)
        x_2= self.conv_2(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.LeakyRelu(x)
        x = self.conv_3(x)
        return self.LeakyRelu(x)
    
        
           

class CLLWCsiNet(nn.Module):
    def __init__(self,reduction=4,residual_num=2):
        super(CLLWCsiNet, self).__init__()
        total_size, in_channel, w, h = 2048, 2, 32, 32
        
        self.encoder_p1 = nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(2,2, [1,7])),
            ("LeakyReLU_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x7",ConvBN(2,2,[7,1])),
            #("LeakyReLU_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ]))
        
        self.encoder_p2= nn.Sequential(OrderedDict([
            ('conv1x3', ConvBN(2, 2, [1, 5])),
            ('LeakyRelu_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv3x1', ConvBN(2, 2, [5, 1])),
            ]))
        
        self.encoder_p3= nn.Sequential(OrderedDict([
            ('conv1x3', ConvBN(2, 2, [1, 3])),
            ('LeakyRelu_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv3x1', ConvBN(2, 2, [3, 1])),
            ]))
        
        self.con1x1=ConvBN(6, 2, [1,7])
        self.LeakyReLU=nn.LeakyReLU(negative_slope=0.3, inplace=True)
        
        ######################## CNN base Laten Space ########################################
        self.encoder_compression=Encoder_Compression()
        
        
        
        self.decoder_get_feedback_in_UE=nn.Sequential(OrderedDict([
            ("first_conv1x7",ConvBN(8,16,[1,7])),
            ("LeakyRelu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv1x7",ConvBN(16,32,[1,7])),
            ("LeakyRelu_2",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x7",ConvBN(32,64,[1,7])),
             #("LeakyRelu_3",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ]))
        
        
        self.remove_AGN=nn.Sequential(OrderedDict([
            ("conv_1x7",ConvBN(8,16,[1,7])),
            ("LeakyReLu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv_7x1",ConvBN(16,32,[1,7])),
            ("LeakyReLu_1",nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv_3x3",ConvBN(32,64,[1,7])),
        ]))
        
        
        
        self.decoder_refine_net = nn.ModuleList([RefineNet(in_channel) for _ in range(residual_num)])
        
        self._last_cov=nn.Sequential(OrderedDict([
            ("firstcov2",ConvBN(2,2,[1,7])),
            ("activation",nn.Sigmoid())
        ]))
        
        
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d)):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        
    # def adding_noise(self,x):
        
    #     signal_power = torch.mean(x**2)

        
    #     SNR_dB = 40

        
    #     SNR_linear = 10**(SNR_dB / 10)

       
    #     noise_power = signal_power / SNR_linear

        
    #     noise = torch.randn_like(x) * torch.sqrt(noise_power)

       
    #     x_noisy = x + noise

    #     return x_noisy
    
    
    
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        #print(x.size())
        x_1=self.encoder_p1(x)
        x_2=self.encoder_p2(x)
        x_3=self.encoder_p3(x)
        x=torch.cat((x_1,x_2,x_3),dim=1)
        x=self.con1x1(x)
        x=self.LeakyReLU(x)
        #print(x.size())
        x = x.contiguous().view(batch_size,64 ,1,32)
        #print(x.size())
        x=self.encoder_compression(x)
        #x_noisy_feedback=self.adding_noise(x)
        #y=self.remove_AGN(x_noisy_feedback)
        x=self.decoder_get_feedback_in_UE(x)
        #x=x-y
        x=self.LeakyReLU(x)
        x = x.contiguous().view(batch_size,2 ,32,32)
        #x = self.refine(x)
        for layer in self.decoder_refine_net:
            x = layer(x)
        x=self._last_cov(x)
       
        
        return x

In [5]:
model=CLLWCsiNet()

In [6]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
pytorch_total_params

66756

In [8]:
import torch
#from Trans_Net.utils import *


state_dict = torch.load(r'/kaggle/input/cllwcsinet-savemodels/CLLWCsiNet-Saved-models/cr-8/outdoor/CLLWCsiNet_model_500_outdoor_256_1x.pth',map_location=torch.device('cpu'))['model_state_dict']

/tmp/ipykernel_31/1965208174.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'/kaggle/input/cllwcsinet-savemodels/CLLWCsiNet-Saved-models/cr-8/o

In [9]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
model.eval()

CLLWCsiNet(
  (encoder_p1): Sequential(
    (first_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (second_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
  )
  (encoder_p2): Sequential(
    (conv1x3): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyRelu_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv3x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)

In [11]:
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # Set quantization config
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8  # Adjust the module types as per your model
)

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [12]:
# For simplicity, I'm using a dummy input for calibration
dummy_input = torch.randn(1, 2, 32, 32)  # Modify according to your input shape
quantized_model(dummy_input)

tensor([[[[5.0060e-01, 3.3311e-01, 7.0319e-01,  ..., 1.2547e-01,
           8.8537e-02, 2.9843e-01],
          [4.1435e-01, 4.7515e-01, 6.3479e-02,  ..., 4.4257e-01,
           4.0133e-01, 3.2340e-01],
          [4.2277e-01, 5.3707e-01, 3.4288e-03,  ..., 1.9133e-01,
           1.1248e-05, 2.4592e-01],
          ...,
          [2.0052e-01, 6.0725e-01, 3.5463e-01,  ..., 1.0220e-02,
           4.3565e-01, 3.5424e-01],
          [7.0683e-01, 5.0142e-01, 7.0482e-01,  ..., 0.0000e+00,
           1.0000e+00, 5.7820e-26],
          [4.0184e-01, 1.6137e-01, 4.3762e-01,  ..., 3.7123e-01,
           4.8528e-01, 5.2594e-01]],

         [[4.9669e-01, 5.9766e-01, 3.3854e-01,  ..., 8.1461e-01,
           8.5017e-01, 6.1591e-01],
          [5.7815e-01, 5.4480e-01, 8.7077e-01,  ..., 5.5363e-01,
           5.6987e-01, 6.3891e-01],
          [5.6144e-01, 5.2940e-01, 9.8094e-01,  ..., 7.7835e-01,
           9.9967e-01, 7.3665e-01],
          ...,
          [7.3324e-01, 4.4880e-01, 6.1998e-01,  ..., 9.6819

In [13]:
quantized_model.to('cpu')

CLLWCsiNet(
  (encoder_p1): Sequential(
    (first_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (second_conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
  )
  (encoder_p2): Sequential(
    (conv1x3): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyRelu_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv3x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)

In [14]:
criterion = nn.MSELoss().to('cpu')

In [15]:
envir = 'outdoor'  # 'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2
img_total = img_height * img_width * img_channels
# network params
#residual_num = 2
encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [16]:
# envir = 'indoor'  # 'indoor' or 'outdoor'
# # image params
# img_height = 32
# img_width = 32
# img_channels = 2
# img_total = img_height * img_width * img_channels
# # network params
# residual_num = 2

# encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32
# Data loading
if envir == 'indoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainin.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalin.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestin.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFin_all.mat')
    X_test = mat['HF_all']  # array

elif envir == 'outdoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainout.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalout.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestout.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFout_all.mat')
    X_test = mat['HF_all']  # array

In [17]:
import numpy as np

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train_length = len(x_train)
x_val_length = len(x_val)
x_test_length = len(x_test)

x_train = np.reshape(x_train, (x_train_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (x_val_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (x_test_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

x_train = torch.tensor(x_train)
x_val = torch.tensor(x_val)
x_test = torch.tensor(x_test)

In [18]:
x_test.shape

torch.Size([20000, 2, 32, 32])

In [19]:
img_height = 32
img_width = 32
img_channels = 2

In [20]:
import math
with torch.no_grad():

    #torch.cuda.empty_cache()
    quantized_model.eval()
    device='cpu'
    x_hat = quantized_model(x_test)
    #torch.quantization.convert(quantized_model, inplace=True)
    x_test = x_test.to('cpu')
    x_hat=x_hat.to('cpu')

    # Calcaulating the NMSE and rho
    # if envir == 'indoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFin_all.mat')
    #     X_test = mat['HF_all']  # array

    # elif envir == 'outdoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFout_all.mat')
    #     X_test = mat['HF_all']  # array

    #X_test = torch.tensor(X_test)
    #X_test = torch.reshape(X_test, (len(X_test), img_height, 125))
    x_test_real = torch.reshape(x_test[:, 0, :, :], (len(x_test), -1))
    x_test_imag = torch.reshape(x_test[:, 1, :, :], (len(x_test), -1))
    x_test_C = x_test_real - 0.5 + 1j * (x_test_imag - 0.5)
    x_hat_real = torch.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
    x_hat_imag = torch.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
    x_hat_C = x_hat_real - 0.5 + 1j * (x_hat_imag - 0.5)
    x_hat_F = torch.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
    X_hat = torch.fft.fft(torch.cat((x_hat_F, torch.zeros((len(x_hat_C), img_height, 257 - img_width))), axis=2), axis=2)
    X_hat = X_hat[:, :, 0:125]

    #n1 = torch.sqrt(torch.sum(torch.conj(X_test) * X_test, axis=1))
    #n2 = torch.sqrt(torch.sum(torch.conj(X_hat) * X_hat, axis=1))
    #aa = abs(torch.sum(torch.conj(X_test) * X_hat, axis=1))
    #rho = torch.mean(aa / (n1 * n2), axis=1)
    X_hat = torch.reshape(X_hat, (len(X_hat), -1))
    #X_test = torch.reshape(X_test, (len(X_test), -1))
    power = torch.sum(abs(x_test_C) ** 2, axis=1)
    power_d = torch.sum(abs(X_hat) ** 2, axis=1)
    mse = torch.sum(abs(x_test_C - x_hat_C) ** 2, axis=1)
    NMSE = 10 * math.log10(torch.mean(mse / power))
    #Correlation = torch.mean(rho).item().real

    # print("In " + envir + " environment")
    # print("When dimension is", encoded_dim)
    print("NMSE is ", NMSE)
    #print("Correlation is ", Correlation)
#
# file = 'CsiNet_' + (envir) + '_dim' + str(encoded_dim) + time.strftime('_%m_%d_%H_%M')
# outfile = "result/result_%s.mat" % file
# savemat(outfile, {'train_loss_history': train_loss_history,
#                   'val_loss_history': val_loss_history,
#                   'training_time': training_time,
#                   'NMSE': NMSE,
#                   'Correlation': Correlation})

NMSE is  -2.462487592576162


In [21]:
# ====== COMPREHENSIVE QUANTIZED MODEL BENCHMARK ======
import time
import torch
from tabulate import tabulate

def benchmark_model(model, input_shape=(2, 32, 32), warmup=10, repetitions=100, batch_sizes=[1, 8, 16, 32, 64], quantized=False):
    """Run comprehensive benchmarks for both quantized and regular models"""
    results = []
    model.eval()
    
    # Prepare model
    if quantized:
        model = torch.quantization.quantize_dynamic(
            model, 
            {torch.nn.Conv2d, torch.nn.Linear},  # Quantize Conv2d layers too
            dtype=torch.qint8
        )
        model_type = "Quantized"
    else:
        model_type = "FP32"
    
    # Benchmark for CPU and GPU (if available and not quantized)
    devices = ['cpu']
    if torch.cuda.is_available() and not quantized:
        devices.append('cuda')
    
    for device in devices:
        # Move model to device
        model.to(device)
        
        # Single sample benchmark
        dummy_input = torch.randn(1, *input_shape).to(device)
        
        # Warm-up
        with torch.no_grad():
            for _ in range(warmup):
                _ = model(dummy_input)
            if device == 'cuda':
                torch.cuda.synchronize()
        
        # Measurement
        start_time = time.time()
        with torch.no_grad():
            for _ in range(repetitions):
                _ = model(dummy_input)
                if device == 'cuda':
                    torch.cuda.synchronize()
        avg_time = (time.time() - start_time) * 1000 / repetitions
        
        results.append([
            f"{device.upper()} ({model_type})",
            "Single",
            f"{avg_time:.2f} ms",
            f"{1000/avg_time:.2f} samples/sec"
        ])
        
        # Batch processing benchmark
        for bs in batch_sizes:
            dummy_batch = torch.randn(bs, *input_shape).to(device)
            
            # Warm-up
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            # Measure
            start = time.time()
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            total_time = (time.time() - start) * 1000
            time_per_sample = total_time / bs
            
            results.append([
                f"{device.upper()} ({model_type})",
                f"Batch {bs}",
                f"{total_time:.2f} ms",
                f"{1000/time_per_sample:.2f} samples/sec"
            ])
    
    return results

# Run benchmarks
print("\n=== Model Benchmark Results ===")
benchmark_results = benchmark_model(model, quantized=False)
quantized_results = benchmark_model(model, quantized=True)

# Combine and print results
all_results = benchmark_results + quantized_results
headers = ["Configuration", "Test Type", "Inference Time", "Throughput"]
print(tabulate(all_results, headers=headers))

# Additional metrics
print("\n=== Quantization Benefits ===")
# Find matching CPU results to compare
fp32_cpu_single = next(r for r in benchmark_results if "CPU (FP32)" in r[0] and "Single" in r[1])
quant_cpu_single = next(r for r in quantized_results if "CPU (Quantized)" in r[0] and "Single" in r[1])

speedup = float(fp32_cpu_single[2].split()[0]) / float(quant_cpu_single[2].split()[0])
print(f"Quantization speedup (CPU single sample): {speedup:.2f}x")

# Model size comparison
def get_model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")/1e6  # in MB
    os.remove("temp.p")
    return size

fp32_size = get_model_size(model)
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Conv2d}, dtype=torch.qint8)
quant_size = get_model_size(quantized_model)

print(f"\nModel size reduction: {fp32_size:.2f}MB -> {quant_size:.2f}MB ({fp32_size/quant_size:.1f}x smaller)")


=== Model Benchmark Results ===
Configuration    Test Type    Inference Time    Throughput
---------------  -----------  ----------------  --------------------
CPU (FP32)       Single       2.95 ms           338.41 samples/sec
CPU (FP32)       Batch 1      2.87 ms           348.34 samples/sec
CPU (FP32)       Batch 8      8.96 ms           892.71 samples/sec
CPU (FP32)       Batch 16     13.28 ms          1204.65 samples/sec
CPU (FP32)       Batch 32     16.69 ms          1916.93 samples/sec
CPU (FP32)       Batch 64     27.57 ms          2321.38 samples/sec
CUDA (FP32)      Single       3.49 ms           286.71 samples/sec
CUDA (FP32)      Batch 1      3.45 ms           290.12 samples/sec
CUDA (FP32)      Batch 8      3.45 ms           2316.98 samples/sec
CUDA (FP32)      Batch 16     3.41 ms           4686.05 samples/sec
CUDA (FP32)      Batch 32     3.46 ms           9259.59 samples/sec
CUDA (FP32)      Batch 64     4.15 ms           15408.73 samples/sec
CPU (Quantized)  Single    